In [ ]:
import pandas as pd

Считывание  сообщений

In [ ]:
sms_path = "SMSSpamCollection.txt"

In [ ]:
sms_spam = pd.read_csv(sms_path, sep='\t',
header=None, names=['Label', 'SMS'])

Вывод SMS и их лейблов

In [ ]:
print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Выше видно, что около 87 % сообщений являются не спамом, а остальные 13 % являются спамом.

Теперь  проанализируем процент спама и не спама сообщений в обучающих и тестовых наборах. Мы ожидаем, что процентные показатели будут близки к тому, что мы имеем в полном наборе данных, где около 87 % сообщений являются не спамом, а остальные 13 % являются спамом.

Удалим знаки препинания и переведем все слова в нижний регистр.

In [ ]:
# До очистки
sms_spam.head(3)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [ ]:
# После очистки
sms_spam['SMS'] = sms_spam['SMS'].str.replace(
   '\W', ' ') # Удаление пунктуации
sms_spam['SMS'] = sms_spam['SMS'].str.lower()
sms_spam.head(3)

,Label,SMS
0,ham,go until jurong point crazy available only ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...


In [ ]:
sms_spam['SMS'] = sms_spam['SMS'].str.split()


Инициируем пустой список 

In [ ]:
vocabulary = []

Используя вложенный цикл, повторяем каждое сообщение в столбце SMS и добавляем каждую строку (слово) в список словарного запаса.

In [ ]:
for sms in sms_spam['SMS']:
   for word in sms:
      vocabulary.append(word)

Преобразуем список словаря в набор с помощью функции set(). Это позволит удалить дубликаты из списка словаря. Затем преобразуем набор слов обратно в список, используя функцию list().

In [ ]:
vocabulary = list(set(vocabulary))

In [ ]:
len(vocabulary)

8753

Создадим словарь, который затем преобразуем в нужный нам фрейм данных.

Начнем с инициализации словаря с именем word_counts_per_sms, где каждый ключ представляет собой уникальное слово (строку) из словаря, а каждое значение представляет собой список длины обучающего набора, где каждый элемент в этом списке равен 0.

In [ ]:
#  Вывод списка длины training_set['SMS']
word_counts_per_sms = {unique_word: [0] * len(sms_spam['SMS']) for unique_word in vocabulary}

Перебираем training_set['SMS'] с помощью функции enumerate(), чтобы получить как индекс, так и SMS-сообщение

Используя вложенный цикл, мы перебираем sms (где sms - это список строк, где каждая строка представляет слово в сообщении).

In [ ]:
for index, sms in enumerate(sms_spam['SMS']):
   for word in sms:
      word_counts_per_sms[word][index] += 1 #увеличиваем индекс на 1

In [ ]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,process,differ,toledo,band,paris,yalrigu,or2stoptxt,0870141701216,3hrs,buy,scream,zoom,optimistic,party,boytoy,wrench,chocolate,bcmsfwc1n3xx,dload,83435,ringing,440,wild,employee,breakfast,freedom,speedchat,norm150p,asthere,unconsciously,l8er,deciding,lately,wither,0789xxxxxxx,mtnl,neville,following,08718738034,arun,...,weed,persons,09066382422,08000930705,blessed,joys,box403,nig,stick,bday,franyxxxxx,waz,off,directors,airport,siguviri,opportunity,says,unbelievable,desperate,ugo,hogolo,pouts,lakhs,thinking,1winawk,yay,peak,hurting,compliments,egg,945,gravity,stuffing,dogg,o,arr,jesus,french,gay
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



Столбец Label отсутствует. С помощью pd.concat() объеденим только что созданный DataFrame с имеющимся набором и получим метки.

In [ ]:
sms_spam_clean = pd.concat([sms_spam, word_counts], axis=1)
sms_spam_clean.head()

,Label,SMS,process,differ,toledo,band,paris,yalrigu,or2stoptxt,0870141701216,3hrs,buy,scream,zoom,optimistic,party,boytoy,wrench,chocolate,bcmsfwc1n3xx,dload,83435,ringing,440,wild,employee,breakfast,freedom,speedchat,norm150p,asthere,unconsciously,l8er,deciding,lately,wither,0789xxxxxxx,mtnl,neville,following,...,weed,persons,09066382422,08000930705,blessed,joys,box403,nig,stick,bday,franyxxxxx,waz,off,directors,airport,siguviri,opportunity,says,unbelievable,desperate,ugo,hogolo,pouts,lakhs,thinking,1winawk,yay,peak,hurting,compliments,egg,945,gravity,stuffing,dogg,o,arr,jesus,french,gay
0,ham,"[go, until, jurong, point, crazy, available, o...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ham,"[ok, lar, joking, wif, u, oni]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Теперь мы проанализируем процент спама и не спама в обучающих и тестовых наборах. Мы ожидаем, что процентные показатели будут близки к тому, что мы имеем в полном наборе данных, где около 87 % сообщений являются не спамом, а остальные 13 % являются спамом.

In [ ]:
# Рандомизируем данные
data_randomized = sms_spam_clean.sample(frac=1, random_state=1)

# Рассчитаем индекс для разделения
training_test_index = round(len(data_randomized) * 0.7)

# Разделим на обучающую и тестовую выборки
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(3900, 8755)
(1672, 8755)


In [ ]:
training_set['Label'].value_counts(normalize=True)

ham     0.865897
spam    0.134103
Name: Label, dtype: float64

In [ ]:
test_set['Label'].value_counts(normalize=True)

ham     0.866029
spam    0.133971
Name: Label, dtype: float64

Удобнее разделить наш набор данных на обучающие и тестовые выборки с помощью метода scikit train_test_split, чтобы мы могли сделать выводы о точности модели на данных, на которых она не была обучена.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sms_spam_clean['SMS'],
                                                    sms_spam_clean['Label'],
                                                    test_size=0.3)

print ("Исходный набор:", sms_spam_clean.shape[0])
print ("Тренировачный набор:", X_train.shape[0])
print ("Тестовый набор:", X_test.shape[0])

Исходный набор: 5572
Тренировачный набор: 3900
Тестовый набор: 1672


Преобразуем набор в матрицу счетчиков токенов.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

In [ ]:
X_train = count_vectorizer.fit_transform(X_train.apply(lambda x: ' '.join(x)))
X_test = count_vectorizer.transform(X_test.apply(lambda x: ' '.join(x)))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
MNB = MultinomialNB()

MNB.fit(X_train, y_train)
y_train_predict = MNB.predict(X_train)
y_predict = MNB.predict(X_test)

print("Результаты для тренировочной выборки")
print(classification_report(y_train, y_train_predict))
print("Результаты для тестовой выборки")
print(classification_report(y_test, y_predict))

Результаты для тренировочной выборки
              precision    recall  f1-score   support

         ham       1.00      1.00      1.00      3383
        spam       0.98      0.97      0.97       517

    accuracy                           0.99      3900
   macro avg       0.99      0.98      0.99      3900
weighted avg       0.99      0.99      0.99      3900

Результаты для тестовой выборки
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1442
        spam       0.96      0.92      0.94       230

    accuracy                           0.98      1672
   macro avg       0.98      0.96      0.97      1672
weighted avg       0.98      0.98      0.98      1672

